In [4]:
import numpy as np
from typing import Optional,Mapping,Sequence,Iterable, Iterator, Tuple, TypeVar, Dict, Callable,List
from rl.markov_decision_process import Policy
import math
from rl.distribution import (Bernoulli, Constant, Categorical, Choose,
                             Distribution, FiniteDistribution)
import numpy as np

from rl.distribution import (Bernoulli, Constant, Categorical, Choose,
                             Distribution, FiniteDistribution)
from dataclasses import dataclass, replace
from rl.markov_decision_process import FinitePolicy, TransitionStep
from rl.function_approx import FunctionApprox

In [5]:
S = TypeVar('S')
A = TypeVar('A')

@dataclass(frozen=True)
class Linear_Approx_Q():
    feature_func: Callable[[S,A],Sequence[float]]
    weight: Sequence[float]

    def update(self, new_weight:Sequence[float]):
        return replace(self,weight = new_weight)

    def evaluate(self,state:S, action:A)->float:
        return np.dot(self.weight,self.feature_func(state,action))

def policy_from_q(
        q: Linear_Approx_Q,
        actions: Mapping[S,Iterable[A]],
        ϵ: float = 0.0
) -> Policy[S, A]:

    explore = Bernoulli(ϵ)

    class QPolicy(Policy[S, A]):
        def act(self, s: S) -> Optional[Distribution[A]]:
            #terminal state?

            if explore.sample():
                return Choose(set(actions))

            ind = np.argmax(q.evaluate([(s, a) for a in actions[s]]))
            return Constant(actions[ind])

    return QPolicy()

def LSPI(feature_func: Callable[[S,A],Sequence[float]],     # feature functions
         simulator: Callable[[S,A],Tuple[S,float]],
         w0: Sequence[float],
         actions: Mapping[S,Iterable[A]],
         states: Sequence[S],
         d: int,
         gamma: float,
         epsilon: float,
         state_distribution: Distribution[S],
         tolerance: float = 1e-6,
         nstop: int = None
         )->Iterator[Sequence[float]]:
    """
    LSTD algorithm.
    feature_func:S->R^d. feature_func(terminal) = 0
    simulator: Take input state and action, output next state and reward
    d: dimension of features
    p0: The initial policy
    w0: R_d, initial weight
    epsilon: small number for initialization A
    actions: allowed actions for each state

    return: Iterator of weights R^d
    """

    # initializations
    A_inverse = 1/epsilon*np.eye(d)
    b = np.zeros(d)
    weight = w0
    q = Linear_Approx_Q(feature_func = feature_func, weight = weight)
    max_steps = round(math.log(tolerance) / math.log(gamma)) if gamma < 1 else nstop

    trace_count = 0

    while True:
        state = state_distribution.sample()
        trace_count += 1
        e2 = 1/trace_count
        # for each step in a episode
        step_count = 0
        while step_count < max_steps:
            step_count += 1
            p = policy_from_q(q,e2,actions)
            action = p.act(state).sample()
            x = feature_func(state,action)
            next_state,reward = simulator(state,action)

            # the off policy next action
            ind = np.argmax([q.evaluate(next_state,action) for action in actions[next_state]])
            ap = actions[next_state][ind]

            xp = feature_func(next_state,ap)

            # update A^(-1), b, weight for each time step. Use Shermannm-Morrison incremental inverse
            v = np.dot(np.transpose(A_inverse),x-gamma*xp)
            A_inverse -= np.outer(np.dot(A_inverse,x),v)/(1+np.dot(v,x))
            b += reward*x
            weight = np.dot(A_inverse,b)

            state = next_state
            q = q.update(new_weight=weight)


        yield q